# Anthropic Claude SDK Tutorial: Building AI Agents for Insurance

This tutorial introduces the **Anthropic Claude SDK** for building AI agents. Unlike high-level frameworks, Anthropic provides a low-level SDK that gives you direct control over Claude's capabilities.

## What You'll Learn

1. Anthropic SDK core concepts: Messages, Tools, and Tool Use
2. Building a Weather Verification Agent with tool calling
3. Building a Claims Eligibility Agent for business logic
4. Manual orchestration of multi-agent pipelines
5. Integrating DSPy for prompt optimization
6. Using MLFlow for experiment tracking

## Prerequisites

- Python 3.10+
- Anthropic API key
- Basic Python knowledge

---

## Why Anthropic SDK?

| Aspect | Anthropic SDK | High-Level Frameworks |
|--------|---------------|----------------------|
| Control | Full control over every API call | Abstractions hide details |
| Flexibility | Build exactly what you need | Constrained by framework patterns |
| Learning Curve | Steeper (must understand API) | Easier (framework handles details) |
| Debugging | Direct access to raw responses | May need to dig through layers |

## 1. Installation & Setup

In [ ]:
# Install Anthropic SDK
# !pip install anthropic

# Additional dependencies
# !pip install httpx beautifulsoup4 python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise ValueError("Please set ANTHROPIC_API_KEY environment variable")

# Model selection
model_name = os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")

print(f"Using model: {model_name}")

## 2. Core Concepts: Anthropic's Tool Use

### 2.1 Messages API

The Anthropic SDK uses a messages-based API:

```python
from anthropic import Anthropic

client = Anthropic()
response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    system="You are a helpful assistant.",
    messages=[{"role": "user", "content": "Hello!"}]
)
```

### 2.2 Tool Definition

Tools are defined as JSON schemas:

```python
tools = [{
    "name": "get_weather",
    "description": "Get weather for a location",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {"type": "string"}
        },
        "required": ["location"]
    }
}]
```

### 2.3 Tool Use Flow

1. Send message with tools
2. Claude responds with `tool_use` block if it wants to call a tool
3. You execute the tool and send back `tool_result`
4. Claude continues with the result

In [ ]:
from anthropic import Anthropic

# Create client
client = Anthropic(api_key=api_key)

print(f"Anthropic client configured")

## 3. Define Tools for Weather Verification

In [ ]:
import httpx
from bs4 import BeautifulSoup
from typing import Dict, Any

# Tool implementations
def geocode_location(location: str) -> Dict[str, Any]:
    """
    Convert location name to coordinates.
    """
    try:
        with httpx.Client() as http_client:
            response = http_client.get(
                "https://nominatim.openstreetmap.org/search",
                params={
                    "q": f"{location}, Australia",
                    "format": "json",
                    "limit": 1
                },
                headers={"User-Agent": "InsuranceWeatherBot/1.0"},
                timeout=10.0
            )
            
            if response.status_code != 200:
                return {"error": f"HTTP {response.status_code}"}
            
            data = response.json()
            if not data:
                return {"error": f"Location not found: {location}"}
            
            result = data[0]
            return {
                "location": result["display_name"],
                "latitude": float(result["lat"]),
                "longitude": float(result["lon"])
            }
    except Exception as e:
        return {"error": str(e)}

def get_bom_weather(latitude: float, longitude: float, date: str) -> Dict[str, Any]:
    """
    Fetch weather from BOM.
    """
    try:
        year, month, day = date.split("-")
        
        url = "https://reg.bom.gov.au/cgi-bin/climate/storms/get_storms.py"
        params = {
            "begin_day": day, "begin_month": month, "begin_year": year,
            "end_day": day, "end_month": month, "end_year": year,
            "lat": latitude, "lng": longitude,
            "event": "all", "distance_from_point": "50", "states": "all"
        }
        
        with httpx.Client() as http_client:
            response = http_client.get(url, params=params, timeout=15.0)
            
            if response.status_code != 200:
                return {"error": f"HTTP {response.status_code}"}
            
            soup = BeautifulSoup(response.text, 'html.parser')
            events = []
            for row in soup.find_all('tr')[1:]:
                cells = row.find_all('td')
                if len(cells) >= 2:
                    event = cells[0].get_text(strip=True)
                    if event:
                        events.append(event)
            
            has_thunder = any('thunder' in e.lower() or 'lightning' in e.lower() for e in events)
            has_wind = any('wind' in e.lower() or 'gust' in e.lower() for e in events)
            
            return {
                "date": date,
                "latitude": latitude,
                "longitude": longitude,
                "events": events,
                "has_thunderstorm": has_thunder,
                "has_strong_wind": has_wind
            }
    except Exception as e:
        return {"error": str(e)}

# Test
print(geocode_location("Brisbane, QLD"))

In [ ]:
# Define tool schemas for Anthropic API
tools = [
    {
        "name": "geocode_location",
        "description": "Convert a location name to latitude/longitude coordinates. Use this before fetching weather data.",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Address or place name (e.g., 'Brisbane, QLD')"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "get_bom_weather",
        "description": "Fetch weather observations from Australian Bureau of Meteorology for a specific location and date.",
        "input_schema": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                    "description": "Latitude coordinate"
                },
                "longitude": {
                    "type": "number",
                    "description": "Longitude coordinate"
                },
                "date": {
                    "type": "string",
                    "description": "Date in YYYY-MM-DD format"
                }
            },
            "required": ["latitude", "longitude", "date"]
        }
    }
]

# Tool dispatch
tool_functions = {
    "geocode_location": geocode_location,
    "get_bom_weather": get_bom_weather
}

print(f"Defined {len(tools)} tools")

## 4. Create the Weather Agent

In [ ]:
import json

WEATHER_AGENT_SYSTEM = """You are a Weather Verification Agent for an Australian insurance company.

Your job is to verify weather conditions for insurance claims by:
1. Using geocode_location to convert addresses to coordinates
2. Using get_bom_weather to fetch weather data from the Bureau of Meteorology

Always provide a structured report including:
- Verified location with coordinates
- Date checked
- Weather events found
- Whether thunderstorms were detected (True/False)
- Whether strong winds were detected (True/False)

Be precise and factual. Only report what the data shows."""

def run_weather_agent(location: str, date: str) -> str:
    """
    Run the weather verification agent with tool use loop.
    """
    
    messages = [
        {
            "role": "user",
            "content": f"Please verify weather conditions for {location} on {date}."
        }
    ]
    
    print(f"\n[Weather Agent] Starting verification for {location} on {date}")
    
    # Tool use loop
    while True:
        response = client.messages.create(
            model=model_name,
            max_tokens=1024,
            system=WEATHER_AGENT_SYSTEM,
            tools=tools,
            messages=messages
        )
        
        # Check if we need to process tool calls
        if response.stop_reason == "tool_use":
            # Add assistant message
            messages.append({
                "role": "assistant",
                "content": response.content
            })
            
            # Process each tool call
            tool_results = []
            for block in response.content:
                if block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    tool_id = block.id
                    
                    print(f"  -> Calling {tool_name}({tool_input})")
                    
                    # Execute tool
                    func = tool_functions.get(tool_name)
                    if func:
                        result = func(**tool_input)
                        print(f"  -> Result: {result}")
                    else:
                        result = {"error": f"Unknown tool: {tool_name}"}
                    
                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": tool_id,
                        "content": json.dumps(result)
                    })
            
            # Add tool results
            messages.append({
                "role": "user",
                "content": tool_results
            })
        else:
            # No more tool calls, get final response
            final_text = ""
            for block in response.content:
                if hasattr(block, 'text'):
                    final_text += block.text
            
            print(f"\n[Weather Agent] Verification complete")
            return final_text

# Test the weather agent
weather_report = run_weather_agent("Brisbane, QLD, 4000", "2025-03-07")
print("\n" + "=" * 60)
print("Weather Report:")
print("=" * 60)
print(weather_report)

## 5. Create the Eligibility Agent

In [ ]:
ELIGIBILITY_AGENT_SYSTEM = """You are a Claims Eligibility Agent for an Australian insurance company.

You receive weather verification reports and determine CAT event eligibility.

## CAT Event Eligibility Rules

**APPROVED** - Qualifies as CAT event if ALL:
- Location is within Australia (lat: -44 to -10, lon: 112 to 154)
- BOTH thunderstorms AND strong winds were detected
- Date is valid (within 90 days, not in future)

**REVIEW** - Needs manual review if:
- Only ONE weather type detected
- Location near Australian borders

**DENIED** - Does not qualify if:
- Neither thunderstorms nor strong winds detected
- Location outside Australia
- Invalid date

## Response Format

Always provide:
1. **DECISION**: APPROVED, REVIEW, or DENIED
2. **REASONING**: Brief explanation
3. **CONFIDENCE**: High, Medium, or Low
4. **RECOMMENDATIONS**: Any follow-up actions"""

def run_eligibility_agent(weather_report: str) -> str:
    """
    Run the eligibility agent (no tools, pure reasoning).
    """
    
    print(f"\n[Eligibility Agent] Analyzing weather report...")
    
    response = client.messages.create(
        model=model_name,
        max_tokens=1024,
        system=ELIGIBILITY_AGENT_SYSTEM,
        messages=[
            {
                "role": "user",
                "content": f"""Please determine CAT event eligibility based on this weather verification report:

{weather_report}

Provide your decision with reasoning."""
            }
        ]
    )
    
    result = ""
    for block in response.content:
        if hasattr(block, 'text'):
            result += block.text
    
    print(f"[Eligibility Agent] Decision complete")
    return result

# Test the eligibility agent
eligibility_result = run_eligibility_agent(weather_report)
print("\n" + "=" * 60)
print("Eligibility Decision:")
print("=" * 60)
print(eligibility_result)

## 6. Complete Pipeline

In [ ]:
def process_claim(location: str, date: str) -> Dict[str, str]:
    """
    Run the complete claims processing pipeline.
    """
    
    print("=" * 60)
    print(f"Processing claim for {location} on {date}")
    print("=" * 60)
    
    # Step 1: Weather Verification
    weather_report = run_weather_agent(location, date)
    
    # Step 2: Eligibility Determination
    eligibility_result = run_eligibility_agent(weather_report)
    
    print("\n" + "=" * 60)
    print("FINAL RESULT:")
    print("=" * 60)
    print(eligibility_result)
    
    return {
        "weather_report": weather_report,
        "eligibility_result": eligibility_result
    }

# Run complete pipeline
result = process_claim("Brisbane, QLD, 4000", "2025-03-07")

## 7. Testing Multiple Claims

In [ ]:
import time

test_claims = [
    ("Brisbane, QLD, 4000", "2025-03-07"),
    ("Sydney, NSW, 2000", "2025-03-07"),
    ("Perth, WA, 6000", "2025-01-15"),
]

def test_all():
    results = []
    for loc, dt in test_claims:
        print(f"\n\n{'#'*60}")
        print(f"# {loc}")
        print(f"{'#'*60}")
        try:
            r = process_claim(loc, dt)
            results.append((loc, dt, "Success", r))
        except Exception as e:
            results.append((loc, dt, "Error", str(e)))
        time.sleep(2)
    return results

# Uncomment to run all
# all_results = test_all()

---

## 8. DSPy Integration for Prompt Optimization

DSPy can optimize the system prompts used with Claude.

In [ ]:
# !pip install dspy

In [ ]:
import dspy

# Configure DSPy with Anthropic
dspy_lm = dspy.LM(
    model=f"anthropic/{model_name}",
    api_key=api_key
)
dspy.configure(lm=dspy_lm)

print("DSPy configured with Anthropic")

In [ ]:
# DSPy signature
class EligibilitySignature(dspy.Signature):
    """Determine CAT event eligibility from weather report."""
    
    weather_report: str = dspy.InputField(desc="Weather verification report")
    decision: str = dspy.OutputField(desc="APPROVED, REVIEW, or DENIED")
    reasoning: str = dspy.OutputField(desc="Explanation for decision")
    confidence: str = dspy.OutputField(desc="High, Medium, or Low")

eligibility_module = dspy.ChainOfThought(EligibilitySignature)

# Test
test_report = """Location: Brisbane, QLD. Lat: -27.47, Lon: 153.02.
Date: 2025-03-07. Events: Thunderstorm, Wind Gust.
Has Thunderstorm: True. Has Strong Wind: True."""

result = eligibility_module(weather_report=test_report)
print(f"Decision: {result.decision}")
print(f"Reasoning: {result.reasoning}")

In [ ]:
# Training examples
examples = [
    dspy.Example(
        weather_report="Brisbane, QLD. -27.47, 153.02. Events: Thunderstorm, Wind Gust. Has Thunderstorm: True. Has Strong Wind: True.",
        decision="APPROVED", reasoning="Both conditions met", confidence="High"
    ).with_inputs("weather_report"),
    
    dspy.Example(
        weather_report="Sydney, NSW. -33.87, 151.21. Events: Light Rain. Has Thunderstorm: False. Has Strong Wind: False.",
        decision="DENIED", reasoning="No severe weather", confidence="High"
    ).with_inputs("weather_report"),
    
    dspy.Example(
        weather_report="Melbourne, VIC. -37.81, 144.96. Events: Thunderstorm. Has Thunderstorm: True. Has Strong Wind: False.",
        decision="REVIEW", reasoning="Only one condition met", confidence="Medium"
    ).with_inputs("weather_report"),
]

# Optimize
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate

def metric(ex, pred, trace=None):
    return ex.decision.upper() == pred.decision.upper()

evaluator = Evaluate(devset=examples, metric=metric, num_threads=1)
baseline = evaluator(eligibility_module)
print(f"Baseline: {baseline}%")

optimizer = BootstrapFewShot(metric=metric, max_bootstrapped_demos=2)
optimized = optimizer.compile(eligibility_module, trainset=examples)

opt_score = evaluator(optimized)
print(f"Optimized: {opt_score}%")

### 8.1 Export Optimized Prompt

In [ ]:
def build_enhanced_system_prompt(module) -> str:
    """Build enhanced system prompt from DSPy module."""
    
    prompt = ELIGIBILITY_AGENT_SYSTEM + "\n\n## Optimized Examples\n"
    
    if hasattr(module, 'demos') and module.demos:
        for i, demo in enumerate(module.demos, 1):
            prompt += f"\n### Example {i}\n"
            for k in ['weather_report', 'decision', 'reasoning']:
                if hasattr(demo, k):
                    v = getattr(demo, k)
                    prompt += f"{k}: {v[:80] if len(str(v)) > 80 else v}\n"
    
    return prompt

enhanced_system = build_enhanced_system_prompt(optimized)
print(enhanced_system[:500] + "...")

In [ ]:
def run_enhanced_eligibility_agent(weather_report: str) -> str:
    """Run eligibility agent with DSPy-optimized prompt."""
    
    response = client.messages.create(
        model=model_name,
        max_tokens=1024,
        system=enhanced_system,
        messages=[{"role": "user", "content": f"Determine eligibility:\n\n{weather_report}"}]
    )
    
    return "".join(b.text for b in response.content if hasattr(b, 'text'))

# Test
enhanced_result = run_enhanced_eligibility_agent(test_report)
print("Enhanced result:")
print(enhanced_result)

---

## 9. MLFlow Integration

In [ ]:
# !pip install mlflow

In [ ]:
import mlflow
from datetime import datetime

mlflow.set_experiment("anthropic-claude-claims")
print(f"MLFlow: {mlflow.get_tracking_uri()}")

In [ ]:
def run_tracked_pipeline(location: str, date: str, use_enhanced: bool = False):
    """Run pipeline with MLFlow tracking."""
    
    run_name = f"claude_{location.split(',')[0]}_{date}"
    if use_enhanced:
        run_name += "_enhanced"
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params({
            "framework": "anthropic",
            "model": model_name,
            "location": location,
            "date": date,
            "use_enhanced": use_enhanced
        })
        
        start = datetime.now()
        
        try:
            weather = run_weather_agent(location, date)
            
            if use_enhanced:
                elig = run_enhanced_eligibility_agent(weather)
            else:
                elig = run_eligibility_agent(weather)
            
            duration = (datetime.now() - start).total_seconds()
            
            decision = "UNKNOWN"
            for d in ["APPROVED", "DENIED", "REVIEW"]:
                if d in elig.upper():
                    decision = d
                    break
            
            mlflow.log_metrics({"duration": duration, "success": 1})
            mlflow.log_text(elig, "result.txt")
            mlflow.set_tags({"decision": decision, "status": "success"})
            
            print(f"\nLogged: {decision}, {duration:.2f}s")
            return {"decision": decision, "duration": duration}
            
        except Exception as e:
            mlflow.log_metrics({"success": 0})
            mlflow.set_tags({"status": "error"})
            raise

# Run
result = run_tracked_pipeline("Brisbane, QLD", "2025-03-07")

In [ ]:
# Compare standard vs enhanced
std = run_tracked_pipeline("Sydney, NSW", "2025-03-07", use_enhanced=False)
enh = run_tracked_pipeline("Sydney, NSW", "2025-03-07", use_enhanced=True)

print(f"Standard: {std}")
print(f"Enhanced: {enh}")

In [ ]:
# View runs
exp = mlflow.get_experiment_by_name("anthropic-claude-claims")
runs = mlflow.search_runs(experiment_ids=[exp.experiment_id])
print(runs[['run_id', 'params.location', 'metrics.duration', 'tags.decision']].to_string())

## 10. Summary & Key Takeaways

### What We Covered

1. **Anthropic SDK**: Low-level control over Claude
2. **Tool Use**: JSON schemas + tool_use/tool_result flow
3. **Manual Orchestration**: Sequential agent pipeline
4. **DSPy Integration**: Optimize system prompts
5. **MLFlow Tracking**: Log experiments

### Strengths

- Direct access to Claude's capabilities
- Full control over every API call
- No framework abstractions to learn
- Easy debugging

### Challenges

- Manual orchestration required
- More code to write
- No built-in multi-agent patterns

### For Insurance Teams

- **Good for**: Teams wanting maximum control, simple pipelines
- **Consider frameworks if**: You need complex multi-agent orchestration

### Next Steps

1. Add streaming for better UX
2. Implement error handling and retries
3. Add more DSPy training examples
4. Compare with framework-based approaches

In [ ]:
print("Tutorial complete!")